In [1]:
import os
import sys
import subprocess
import datetime as dt
import itertools
import numpy as np

import xarray as xr

In [59]:
# define leadtimes to extract (as indices to daily files, so 0 is 1/2 day lead)
def agg_l(xl):
    for x in xl:
        if x<30:
            yield x
        elif x<90 and x%15==0:
            yield x
        elif x%30==0:
            yield x

leads=[el for el in agg_l(range(0,215))]
print(' '.join(str(el) for el in leads))
print(len(leads))

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 45 60 75 90 120 150 180 210
39


In [3]:
mdir='/space/hall5/sitestore/eccc/crd/ccrn/users/reo000/data/predictions/cansipsv3_daily/CanESM5'
#fnameCanESMjoined=lambda mdir, yyyy, mm, dd, hh:f"{mdir}/joined/cwao_CanESM5.1p1bc-v20240611_hindcast_S{yyyy:04}{mm:02}{dd:02}{hh:02}_ocean_6hr_surface_tso.nc"
fnameCanESMdaily=lambda mdir, yyyy, mm, dd, hh:f"{mdir}/joined/cwao_CanESM5.1p1bc-v20240611_hindcast_S{yyyy:04}{mm:02}{dd:02}{hh:02}_ocean_1d_surface_tso.nc"
fnameCanESMAnom=lambda mdir, yyyy, mm, dd, hh:f"{mdir}/joined/cwao_CanESM5.1p1bc-v20240611_hindcast_S{yyyy:04}{mm:02}{dd:02}{hh:02}_ocean_1d_surface_tso.nc"

In [21]:
#for mm in range(1,13): month loop
mm=1

dd=1
hh=0
flist=[fnameCanESMdaily(mdir,yyyy,mm,dd,hh) for yyyy in range(1993,2025) if yyyy<2024 or mm<=6] # stop at Jul 2024

flist

['/space/hall5/sitestore/eccc/crd/ccrn/users/reo000/data/predictions/cansipsv3_daily/CanESM5/joined/cwao_CanESM5.1p1bc-v20240611_hindcast_S1993010100_ocean_1d_surface_tso.nc',
 '/space/hall5/sitestore/eccc/crd/ccrn/users/reo000/data/predictions/cansipsv3_daily/CanESM5/joined/cwao_CanESM5.1p1bc-v20240611_hindcast_S1994010100_ocean_1d_surface_tso.nc',
 '/space/hall5/sitestore/eccc/crd/ccrn/users/reo000/data/predictions/cansipsv3_daily/CanESM5/joined/cwao_CanESM5.1p1bc-v20240611_hindcast_S1995010100_ocean_1d_surface_tso.nc',
 '/space/hall5/sitestore/eccc/crd/ccrn/users/reo000/data/predictions/cansipsv3_daily/CanESM5/joined/cwao_CanESM5.1p1bc-v20240611_hindcast_S1996010100_ocean_1d_surface_tso.nc',
 '/space/hall5/sitestore/eccc/crd/ccrn/users/reo000/data/predictions/cansipsv3_daily/CanESM5/joined/cwao_CanESM5.1p1bc-v20240611_hindcast_S1997010100_ocean_1d_surface_tso.nc',
 '/space/hall5/sitestore/eccc/crd/ccrn/users/reo000/data/predictions/cansipsv3_daily/CanESM5/joined/cwao_CanESM5.1p1bc-v

In [53]:
ff=xr.open_mfdataset(flist,parallel=True,combine='nested',concat_dim='reftime',chunks={'reftime':-1,'leadtime':5,'r':-1,'lat':-1,'lon':-1})

In [54]:
ff

<xarray.Dataset> Size: 71GB
Dimensions:   (reftime: 32, lat: 180, bnds: 2, lon: 360, r: 20, leadtime: 215)
Coordinates:
  * lat       (lat) float64 1kB -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * leadtime  (leadtime) timedelta64[ns] 2kB 0 days 15:00:00 ... 214 days 15:...
  * lon       (lon) float64 3kB 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
  * reftime   (reftime) datetime64[ns] 256B 1993-01-01 1994-01-01 ... 2024-01-01
    time      (reftime, leadtime) datetime64[ns] 55kB dask.array<chunksize=(1, 5), meta=np.ndarray>
Dimensions without coordinates: bnds, r
Data variables:
    lat_bnds  (reftime, lat, bnds) float64 92kB dask.array<chunksize=(1, 180, 2), meta=np.ndarray>
    lon_bnds  (reftime, lon, bnds) float64 184kB dask.array<chunksize=(1, 360, 2), meta=np.ndarray>
    tso       (reftime, r, leadtime, lat, lon) float64 71GB dask.array<chunksize=(1, 20, 5, 180, 360), meta=np.ndarray>
Attributes: (12/21)
    comment:                  Produced by ECCC (Environment and Climate Chang...
    summary:                  Seasonal Forecast data produced by ECCC as its ...
    title:                    ECCC seasonal forecast model output prepared fo...
    forecast_type:            hindcast
    Conventions:              CF-1.6 C3S-0.1
    project:                  C3S Seasonal Forecast
    ...                       ...
    forecast_reference_time:  1993-01-01T00:00:00Z
    history:                  Sat Aug 31 04:30:30 2024: ncecat /space/hall5/s...
    data_licence:             1) GRANT OF LICENCE - The Government of Canada ...
    commit:                    
    creation_date:            2024-07-07T07:30:39Z
    NCO:                      netCDF Operators version 5.0.5 (Homepage = http...

In [55]:
ff.tso

<xarray.DataArray 'tso' (reftime: 32, r: 20, leadtime: 215, lat: 180, lon: 360)> Size: 71GB
dask.array<concatenate, shape=(32, 20, 215, 180, 360), dtype=float64, chunksize=(1, 20, 5, 180, 360), chunktype=numpy.ndarray>
Coordinates:
  * lat       (lat) float64 1kB -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * leadtime  (leadtime) timedelta64[ns] 2kB 0 days 15:00:00 ... 214 days 15:...
  * lon       (lon) float64 3kB 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
  * reftime   (reftime) datetime64[ns] 256B 1993-01-01 1994-01-01 ... 2024-01-01
    time      (reftime, leadtime) datetime64[ns] 55kB dask.array<chunksize=(1, 5), meta=np.ndarray>
Dimensions without coordinates: r
Attributes:
    cell_methods:   leadtime: point
    grid_mapping:   hcrs
    long_name:      Sea Surface Temperature
    standard_name:  sea_surface_temperature
    units:          K

In [57]:
EClim=ff.tso.mean(dim='r').mean(dim='reftime')

In [ ]:
EClim2=ff.tso.sel(reftime>=np.datetime64()).mean(dim='r').mean(dim='reftime')

In [77]:
np.datetime64('1993-01-01')

numpy.datetime64('1993-01-01')

In [81]:
test=ff.tso.sel(reftime=slice(np.datetime64('1993-01-01'),np.datetime64('1994-12-31')))

In [82]:
test.reftime

<xarray.DataArray 'reftime' (reftime: 2)> Size: 16B
array(['1993-01-01T00:00:00.000000000', '1994-01-01T00:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * reftime  (reftime) datetime64[ns] 16B 1993-01-01 1994-01-01
Attributes:
    standard_name:  forecast_reference_time
    long_name:      Start date of the forecast

In [58]:
EClim

<xarray.DataArray 'tso' (leadtime: 215, lat: 180, lon: 360)> Size: 111MB
dask.array<mean_agg-aggregate, shape=(215, 180, 360), dtype=float64, chunksize=(5, 180, 360), chunktype=numpy.ndarray>
Coordinates:
  * lat       (lat) float64 1kB -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * leadtime  (leadtime) timedelta64[ns] 2kB 0 days 15:00:00 ... 214 days 15:...
  * lon       (lon) float64 3kB 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5

In [60]:
Anom0=ff.tso-EClim

In [64]:
66.43/43

1.5448837209302326

In [61]:
Anom0

<xarray.DataArray 'tso' (reftime: 32, r: 20, leadtime: 215, lat: 180, lon: 360)> Size: 71GB
dask.array<sub, shape=(32, 20, 215, 180, 360), dtype=float64, chunksize=(1, 20, 5, 180, 360), chunktype=numpy.ndarray>
Coordinates:
  * lat       (lat) float64 1kB -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * leadtime  (leadtime) timedelta64[ns] 2kB 0 days 15:00:00 ... 214 days 15:...
  * lon       (lon) float64 3kB 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
  * reftime   (reftime) datetime64[ns] 256B 1993-01-01 1994-01-01 ... 2024-01-01
    time      (reftime, leadtime) datetime64[ns] 55kB dask.array<chunksize=(1, 5), meta=np.ndarray>
Dimensions without coordinates: r

In [65]:
int(len(ff.leadtime.values)/5)

43

In [68]:
test=Anom0.isel(leadtime=slice(5,10))

In [70]:
Anom0.leadtime[:10]

<xarray.DataArray 'leadtime' (leadtime: 10)> Size: 80B
array([ 54000000000000, 140400000000000, 226800000000000, 313200000000000,
       399600000000000, 486000000000000, 572400000000000, 658800000000000,
       745200000000000, 831600000000000], dtype='timedelta64[ns]')
Coordinates:
  * leadtime  (leadtime) timedelta64[ns] 80B 0 days 15:00:00 ... 9 days 15:00:00
Attributes:
    long_name:      Time elapsed since the start of the forecast
    standard_name:  forecast_period

In [69]:
test.leadtime

<xarray.DataArray 'leadtime' (leadtime: 5)> Size: 40B
array([486000000000000, 572400000000000, 658800000000000, 745200000000000,
       831600000000000], dtype='timedelta64[ns]')
Coordinates:
  * leadtime  (leadtime) timedelta64[ns] 40B 5 days 15:00:00 ... 9 days 15:00:00
Attributes:
    long_name:      Time elapsed since the start of the forecast
    standard_name:  forecast_period

In [71]:
%%time
p0='/space/hall5/sitestore/eccc/crd/ccrn/users/reo000/data/predictions/cansipsv3_daily/CanESM5/joined'
for ix in range(0,int(len(ff.leadtime.values)/5)):
    fname=f"test_l_{5*ix}_{5*(ix+1)-1}.nc"
    temp=Anom0.isel(leadtime=slice(5*ix,5*(ix+1)))
    temp.to_netcdf(os.path.join(p0,fname),mode='w')

CPU times: user 39min 13s, sys: 2min 14s, total: 41min 28s
Wall time: 53min 49s


In [18]:
ff.reftime

<xarray.DataArray 'reftime' (reftime: 32)> Size: 256B
array(['1993-01-01T00:00:00.000000000', '1994-01-01T00:00:00.000000000',
       '1995-01-01T00:00:00.000000000', '1996-01-01T00:00:00.000000000',
       '1997-01-01T00:00:00.000000000', '1998-01-01T00:00:00.000000000',
       '1999-01-01T00:00:00.000000000', '2000-01-01T00:00:00.000000000',
       '2001-01-01T00:00:00.000000000', '2002-01-01T00:00:00.000000000',
       '2003-01-01T00:00:00.000000000', '2004-01-01T00:00:00.000000000',
       '2005-01-01T00:00:00.000000000', '2006-01-01T00:00:00.000000000',
       '2007-01-01T00:00:00.000000000', '2008-01-01T00:00:00.000000000',
       '2009-01-01T00:00:00.000000000', '2010-01-01T00:00:00.000000000',
       '2011-01-01T00:00:00.000000000', '2012-01-01T00:00:00.000000000',
       '2013-01-01T00:00:00.000000000', '2014-01-01T00:00:00.000000000',
       '2015-01-01T00:00:00.000000000', '2016-01-01T00:00:00.000000000',
       '2017-01-01T00:00:00.000000000', '2018-01-01T00:00:00.000000000',
       '2019-01-01T00:00:00.000000000', '2020-01-01T00:00:00.000000000',
       '2021-01-01T00:00:00.000000000', '2022-01-01T00:00:00.000000000',
       '2023-01-01T00:00:00.000000000', '2024-01-01T00:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * reftime  (reftime) datetime64[ns] 256B 1993-01-01 1994-01-01 ... 2024-01-01
Attributes:
    standard_name:  forecast_reference_time
    long_name:      Start date of the forecast

In [10]:
# check daily file:
p0='/space/hall5/sitestore/eccc/crd/ccrn/users/reo000/data/predictions/cansipsv3_daily/CanESM5/joined'
fn='cwao_CanESM5.1p1bc-v20240611_hindcast_S2024060100_ocean_1d_surface_tso.nc'
ff=xr.open_dataset(os.path.join(p0,fn))

In [11]:
ff

<xarray.Dataset> Size: 2GB
Dimensions:   (lat: 180, bnds: 2, lon: 360, r: 20, leadtime: 215)
Coordinates:
  * lat       (lat) float64 1kB -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * leadtime  (leadtime) timedelta64[ns] 2kB 0 days 15:00:00 ... 214 days 15:...
  * lon       (lon) float64 3kB 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
    reftime   datetime64[ns] 8B ...
    time      (leadtime) datetime64[ns] 2kB ...
Dimensions without coordinates: bnds, r
Data variables:
    lat_bnds  (lat, bnds) float64 3kB ...
    lon_bnds  (lon, bnds) float64 6kB ...
    tso       (r, leadtime, lat, lon) float64 2GB ...
Attributes: (12/21)
    comment:                  Produced by ECCC (Environment and Climate Chang...
    summary:                  Seasonal Forecast data produced by ECCC as its ...
    title:                    ECCC seasonal forecast model output prepared fo...
    forecast_type:            hindcast
    Conventions:              CF-1.6 C3S-0.1
    project:                  C3S Seasonal Forecast
    ...                       ...
    forecast_reference_time:  2024-06-01T00:00:00Z
    history:                  Sat Aug 31 21:23:21 2024: ncecat /space/hall5/s...
    data_licence:             1) GRANT OF LICENCE - The Government of Canada ...
    commit:                    
    creation_date:            2024-07-16T02:00:16Z
    NCO:                      netCDF Operators version 5.0.5 (Homepage = http...

In [13]:
ff.reftime.values

numpy.datetime64('2024-06-01T00:00:00.000000000')

In [7]:
EM=ff.mean(dim='r')

In [9]:
EM.tso

<xarray.DataArray 'tso' (leadtime: 215, lat: 180, lon: 360)> Size: 111MB
array([[[         nan,          nan,          nan, ...,          nan,
                  nan,          nan],
        [         nan,          nan,          nan, ...,          nan,
                  nan,          nan],
        [         nan,          nan,          nan, ...,          nan,
                  nan,          nan],
        ...,
        [271.39637146, 271.39658928, 271.39688148, ..., 271.39990921,
         271.39768944, 271.3962307 ],
        [271.42938576, 271.42987556, 271.43037376, ..., 271.42793922,
         271.42841606, 271.42889786],
        [271.46108322, 271.46073608, 271.46039085, ..., 271.46215668,
         271.46179352, 271.46143799]],

       [[         nan,          nan,          nan, ...,          nan,
                  nan,          nan],
        [         nan,          nan,          nan, ...,          nan,
                  nan,          nan],
        [         nan,          nan,          nan, ...,          nan,
                  nan,          nan],
...
        [271.42592125, 271.42469292, 271.42348862, ..., 271.42969933,
         271.42843246, 271.427174  ],
        [271.46283302, 271.46237526, 271.46193161, ..., 271.46426735,
         271.46377678, 271.46330109],
        [271.4929409 , 271.49286308, 271.49278374, ..., 271.49317856,
         271.49309883, 271.49302101]],

       [[         nan,          nan,          nan, ...,          nan,
                  nan,          nan],
        [         nan,          nan,          nan, ...,          nan,
                  nan,          nan],
        [         nan,          nan,          nan, ...,          nan,
                  nan,          nan],
        ...,
        [271.42617569, 271.42494545, 271.42374382, ..., 271.42995377,
         271.42868767, 271.42742958],
        [271.46260834, 271.46214867, 271.46170044, ..., 271.4640522 ,
         271.46355782, 271.46307793],
        [271.49277229, 271.4927002 , 271.49263191, ..., 271.49298401,
         271.4929142 , 271.49284325]]])
Coordinates:
  * lat       (lat) float64 1kB -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * leadtime  (leadtime) timedelta64[ns] 2kB 0 days 15:00:00 ... 214 days 15:...
  * lon       (lon) float64 3kB 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
    reftime   datetime64[ns] 8B ...
    time      (leadtime) datetime64[ns] 2kB ...

In [14]:
ff.close()

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 45,
 60,
 75,
 90,
 120,
 150,
 180,
 210]

In [53]:
dd=1
hh=0
for yyyy in range(1993,2025):
    for mm in range(1,13):
        if yyyy==2024 and mm>6:
            pass
        else:
            fconvert(yyyy,mm,dd,hh)
            fin=fnameCanESMjoined(mdir,yyyy,mm,dd,hh)
            fout=fnameCanESMjoined(mdir,yyyy,mm,dd,hh)
            print(fin[70:])
            print(fout[70:])
            print('\n')

sipsv3_daily/CanESM5/joined/cwao_CanESM5.1p1bc-v20240611_hindcast_S1993010100_ocean_6hr_surface_tso.nc
sipsv3_daily/CanESM5/joined/cwao_CanESM5.1p1bc-v20240611_hindcast_S1993010100_ocean_6hr_surface_tso.nc


sipsv3_daily/CanESM5/joined/cwao_CanESM5.1p1bc-v20240611_hindcast_S1993020100_ocean_6hr_surface_tso.nc
sipsv3_daily/CanESM5/joined/cwao_CanESM5.1p1bc-v20240611_hindcast_S1993020100_ocean_6hr_surface_tso.nc


sipsv3_daily/CanESM5/joined/cwao_CanESM5.1p1bc-v20240611_hindcast_S1993030100_ocean_6hr_surface_tso.nc
sipsv3_daily/CanESM5/joined/cwao_CanESM5.1p1bc-v20240611_hindcast_S1993030100_ocean_6hr_surface_tso.nc


sipsv3_daily/CanESM5/joined/cwao_CanESM5.1p1bc-v20240611_hindcast_S1993040100_ocean_6hr_surface_tso.nc
sipsv3_daily/CanESM5/joined/cwao_CanESM5.1p1bc-v20240611_hindcast_S1993040100_ocean_6hr_surface_tso.nc


sipsv3_daily/CanESM5/joined/cwao_CanESM5.1p1bc-v20240611_hindcast_S1993050100_ocean_6hr_surface_tso.nc
sipsv3_daily/CanESM5/joined/cwao_CanESM5.1p1bc-v20240611_hindcast

In [60]:
fff='/space/hall5/sitestore/eccc/crd/ccrn/users/reo000/data/predictions/cansipsv3_daily/CanESM5/joined/cwao_CanESM5.1p1bc-v20240611_hindcast_S1993010100_ocean_6hr_surface_tso.nc'

In [62]:
ff=xr.open_dataset(fff,decode_times=False):q


In [33]:
ff2=ff.drop_vars(['realization','hcrs']).rename({'record':'r'})

In [30]:
ff3=ff2.coarsen(leadtime=4).mean()

In [31]:
ff3

<xarray.Dataset> Size: 2GB
Dimensions:   (lat: 180, bnds: 2, lon: 360, record: 20, leadtime: 215)
Coordinates:
  * lat       (lat) float64 1kB -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * leadtime  (leadtime) float64 2kB 15.0 39.0 63.0 ... 5.127e+03 5.151e+03
  * lon       (lon) float64 3kB 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
    reftime   float64 8B 0.0
    time      (leadtime) float64 2kB 15.0 39.0 63.0 ... 5.127e+03 5.151e+03
Dimensions without coordinates: bnds, record
Data variables:
    lat_bnds  (lat, bnds) float64 3kB -90.0 -89.0 -89.0 -88.0 ... 89.0 89.0 90.0
    lon_bnds  (lon, bnds) float64 6kB 0.0 1.0 1.0 2.0 ... 359.0 359.0 360.0
    tso       (record, leadtime, lat, lon) float64 2GB nan nan ... 271.5 271.5
Attributes: (12/21)
    comment:                  Produced by ECCC (Environment and Climate Chang...
    summary:                  Seasonal Forecast data produced by ECCC as its ...
    title:                    ECCC seasonal forecast model output prepared fo...
    forecast_type:            hindcast
    Conventions:              CF-1.6 C3S-0.1
    project:                  C3S Seasonal Forecast
    ...                       ...
    forecast_reference_time:  1993-01-01T00:00:00Z
    history:                  Sat Aug 31 04:30:30 2024: ncecat /space/hall5/s...
    data_licence:             1) GRANT OF LICENCE - The Government of Canada ...
    commit:                    
    creation_date:            2024-07-07T07:30:39Z
    NCO:                      netCDF Operators version 5.0.5 (Homepage = http...

In [35]:
ff3.leadtime/24

<xarray.DataArray 'leadtime' (leadtime: 215)> Size: 2kB
array([  0.625,   1.625,   2.625,   3.625,   4.625,   5.625,   6.625,
         7.625,   8.625,   9.625,  10.625,  11.625,  12.625,  13.625,
        14.625,  15.625,  16.625,  17.625,  18.625,  19.625,  20.625,
        21.625,  22.625,  23.625,  24.625,  25.625,  26.625,  27.625,
        28.625,  29.625,  30.625,  31.625,  32.625,  33.625,  34.625,
        35.625,  36.625,  37.625,  38.625,  39.625,  40.625,  41.625,
        42.625,  43.625,  44.625,  45.625,  46.625,  47.625,  48.625,
        49.625,  50.625,  51.625,  52.625,  53.625,  54.625,  55.625,
        56.625,  57.625,  58.625,  59.625,  60.625,  61.625,  62.625,
        63.625,  64.625,  65.625,  66.625,  67.625,  68.625,  69.625,
        70.625,  71.625,  72.625,  73.625,  74.625,  75.625,  76.625,
        77.625,  78.625,  79.625,  80.625,  81.625,  82.625,  83.625,
        84.625,  85.625,  86.625,  87.625,  88.625,  89.625,  90.625,
        91.625,  92.625,  93.625,  94.625,  95.625,  96.625,  97.625,
        98.625,  99.625, 100.625, 101.625, 102.625, 103.625, 104.625,
       105.625, 106.625, 107.625, 108.625, 109.625, 110.625, 111.625,
       112.625, 113.625, 114.625, 115.625, 116.625, 117.625, 118.625,
       119.625, 120.625, 121.625, 122.625, 123.625, 124.625, 125.625,
       126.625, 127.625, 128.625, 129.625, 130.625, 131.625, 132.625,
       133.625, 134.625, 135.625, 136.625, 137.625, 138.625, 139.625,
       140.625, 141.625, 142.625, 143.625, 144.625, 145.625, 146.625,
       147.625, 148.625, 149.625, 150.625, 151.625, 152.625, 153.625,
       154.625, 155.625, 156.625, 157.625, 158.625, 159.625, 160.625,
       161.625, 162.625, 163.625, 164.625, 165.625, 166.625, 167.625,
       168.625, 169.625, 170.625, 171.625, 172.625, 173.625, 174.625,
       175.625, 176.625, 177.625, 178.625, 179.625, 180.625, 181.625,
       182.625, 183.625, 184.625, 185.625, 186.625, 187.625, 188.625,
       189.625, 190.625, 191.625, 192.625, 193.625, 194.625, 195.625,
       196.625, 197.625, 198.625, 199.625, 200.625, 201.625, 202.625,
       203.625, 204.625, 205.625, 206.625, 207.625, 208.625, 209.625,
       210.625, 211.625, 212.625, 213.625, 214.625])
Coordinates:
  * leadtime  (leadtime) float64 2kB 15.0 39.0 63.0 ... 5.127e+03 5.151e+03
    reftime   float64 8B 0.0
    time      (leadtime) float64 2kB 15.0 39.0 63.0 ... 5.127e+03 5.151e+03

In [44]:
ff3.tso.assign_attrs({'postprocess':'daily time average, [(6,12,18,24),...]'})

<xarray.DataArray 'tso' (record: 20, leadtime: 215, lat: 180, lon: 360)> Size: 2GB
array([[[[         nan,          nan,          nan, ...,          nan,
                   nan,          nan],
         [         nan,          nan,          nan, ...,          nan,
                   nan,          nan],
         [         nan,          nan,          nan, ...,          nan,
                   nan,          nan],
         ...,
         [271.35134125, 271.35227203, 271.35319519, ..., 271.34988403,
          271.3500061 , 271.35040283],
         [271.36029816, 271.36016846, 271.36001587, ..., 271.36067963,
          271.36056519, 271.36042786],
         [271.38357544, 271.38339233, 271.38319397, ..., 271.38417053,
          271.3839798 , 271.38376617]],

        [[         nan,          nan,          nan, ...,          nan,
                   nan,          nan],
         [         nan,          nan,          nan, ...,          nan,
                   nan,          nan],
         [         nan,          nan,          nan, ...,          nan,
                   nan,          nan],
...
         [271.3717804 , 271.36997986, 271.36816406, ..., 271.3789444 ,
          271.37609863, 271.37358856],
         [271.44255066, 271.44052887, 271.43852997, ..., 271.44866943,
          271.44660187, 271.44456482],
         [271.49517822, 271.49465179, 271.49411774, ..., 271.49681091,
          271.49626923, 271.49571991]],

        [[         nan,          nan,          nan, ...,          nan,
                   nan,          nan],
         [         nan,          nan,          nan, ...,          nan,
                   nan,          nan],
         [         nan,          nan,          nan, ...,          nan,
                   nan,          nan],
         ...,
         [271.37606812, 271.37414551, 271.37221527, ..., 271.38349915,
          271.38059998, 271.37799835],
         [271.44705963, 271.44511414, 271.44316864, ..., 271.45301056,
          271.45100403, 271.44902802],
         [271.49867249, 271.4981842 , 271.49768829, ..., 271.50016022,
          271.49966431, 271.49916077]]]])
Coordinates:
  * lat       (lat) float64 1kB -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * leadtime  (leadtime) float64 2kB 15.0 39.0 63.0 ... 5.127e+03 5.151e+03
  * lon       (lon) float64 3kB 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
    reftime   float64 8B 0.0
    time      (leadtime) float64 2kB 15.0 39.0 63.0 ... 5.127e+03 5.151e+03
Dimensions without coordinates: record
Attributes:
    cell_methods:   leadtime: point
    grid_mapping:   hcrs
    long_name:      Sea Surface Temperature
    standard_name:  sea_surface_temperature
    units:          K
    postprocess:    daily time average, [(6,12,18,24),...]

In [38]:
ff3.tso

<xarray.DataArray 'tso' (record: 20, leadtime: 215, lat: 180, lon: 360)> Size: 2GB
array([[[[         nan,          nan,          nan, ...,          nan,
                   nan,          nan],
         [         nan,          nan,          nan, ...,          nan,
                   nan,          nan],
         [         nan,          nan,          nan, ...,          nan,
                   nan,          nan],
         ...,
         [271.35134125, 271.35227203, 271.35319519, ..., 271.34988403,
          271.3500061 , 271.35040283],
         [271.36029816, 271.36016846, 271.36001587, ..., 271.36067963,
          271.36056519, 271.36042786],
         [271.38357544, 271.38339233, 271.38319397, ..., 271.38417053,
          271.3839798 , 271.38376617]],

        [[         nan,          nan,          nan, ...,          nan,
                   nan,          nan],
         [         nan,          nan,          nan, ...,          nan,
                   nan,          nan],
         [         nan,          nan,          nan, ...,          nan,
                   nan,          nan],
...
         [271.3717804 , 271.36997986, 271.36816406, ..., 271.3789444 ,
          271.37609863, 271.37358856],
         [271.44255066, 271.44052887, 271.43852997, ..., 271.44866943,
          271.44660187, 271.44456482],
         [271.49517822, 271.49465179, 271.49411774, ..., 271.49681091,
          271.49626923, 271.49571991]],

        [[         nan,          nan,          nan, ...,          nan,
                   nan,          nan],
         [         nan,          nan,          nan, ...,          nan,
                   nan,          nan],
         [         nan,          nan,          nan, ...,          nan,
                   nan,          nan],
         ...,
         [271.37606812, 271.37414551, 271.37221527, ..., 271.38349915,
          271.38059998, 271.37799835],
         [271.44705963, 271.44511414, 271.44316864, ..., 271.45301056,
          271.45100403, 271.44902802],
         [271.49867249, 271.4981842 , 271.49768829, ..., 271.50016022,
          271.49966431, 271.49916077]]]])
Coordinates:
  * lat       (lat) float64 1kB -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * leadtime  (leadtime) float64 2kB 15.0 39.0 63.0 ... 5.127e+03 5.151e+03
  * lon       (lon) float64 3kB 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
    reftime   float64 8B 0.0
    time      (leadtime) float64 2kB 15.0 39.0 63.0 ... 5.127e+03 5.151e+03
Dimensions without coordinates: record
Attributes:
    cell_methods:   leadtime: point
    grid_mapping:   hcrs
    long_name:      Sea Surface Temperature
    standard_name:  sea_surface_temperature
    units:          K

In [ ]:
# next extract series of 

In [39]:
ff='/space/hall5/siffore/eccc/crd/ccrn/users/reo000/data/predictions/cansipsv3_daily/CanESM5/cwao_CanESM5.1p1bc-v20240611_hindcast_S1993010100_ocean_6hr_surface_tso_r01i00p00.nc'

In [40]:
file=xr.open_dataset(ff)

In [41]:
file

<xarray.Dataset> Size: 446MB
Dimensions:      (lat: 180, bnds: 2, leadtime: 860, lon: 360)
Coordinates:
  * lat          (lat) float64 1kB -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
  * leadtime     (leadtime) timedelta64[ns] 7kB 0 days 06:00:00 ... 215 days ...
  * lon          (lon) float64 3kB 0.5 1.5 2.5 3.5 ... 356.5 357.5 358.5 359.5
    realization  |S31 31B ...
    reftime      datetime64[ns] 8B ...
    time         (leadtime) datetime64[ns] 7kB ...
Dimensions without coordinates: bnds
Data variables:
    hcrs         |S1 1B ...
    lat_bnds     (lat, bnds) float64 3kB ...
    lon_bnds     (lon, bnds) float64 6kB ...
    tso          (leadtime, lat, lon) float64 446MB ...
Attributes: (12/20)
    comment:                  Produced by ECCC (Environment and Climate Chang...
    summary:                  Seasonal Forecast data produced by ECCC as its ...
    title:                    ECCC seasonal forecast model output prepared fo...
    forecast_type:            hindcast
    Conventions:              CF-1.6 C3S-0.1
    project:                  C3S Seasonal Forecast
    ...                       ...
    frequency:                6hr
    forecast_reference_time:  1993-01-01T00:00:00Z
    history:                   
    data_licence:             1) GRANT OF LICENCE - The Government of Canada ...
    commit:                    
    creation_date:            2024-07-07T07:30:39Z

In [42]:
file.tso

<xarray.DataArray 'tso' (leadtime: 860, lat: 180, lon: 360)> Size: 446MB
[55728000 values with dtype=float64]
Coordinates:
  * lat          (lat) float64 1kB -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
  * leadtime     (leadtime) timedelta64[ns] 7kB 0 days 06:00:00 ... 215 days ...
  * lon          (lon) float64 3kB 0.5 1.5 2.5 3.5 ... 356.5 357.5 358.5 359.5
    realization  |S31 31B ...
    reftime      datetime64[ns] 8B ...
    time         (leadtime) datetime64[ns] 7kB ...
Attributes:
    cell_methods:   leadtime: point
    grid_mapping:   hcrs
    long_name:      Sea Surface Temperature
    standard_name:  sea_surface_temperature
    units:          K

In [54]:
test3.close()

In [55]:
test2.close()

In [56]:
test.close()

AttributeError: 'list' object has no attribute 'close'

In [ ]:
ff3.to_netcdf()

In [63]:
180/30*360/30

72.0

In [64]:
72/2

36.0